# Reverse CNN
This puzzle is essentially a reverse CNN.  The goal it to find the number of possible previous states given an array that underwent a 2x2 sliding window transformation (CNN) in which a 1 is produced if there is only 1 square of the 4 that contains a 1.  All other possibilities result in a 0.  I start in the lower right of the array and work backwards to produce all possible solutions.  This program is meant to be run on arrays no larger than 9x50

In [1]:
false = [(0,0,0,0),
         (1,1,1,1),
         (1,1,0,0),
         (1,0,1,0),
         (0,0,1,1),
         (0,1,0,1),
         (1,0,0,1),
         (0,1,1,0),
         (1,1,1,0),
         (1,1,0,1),
         (1,0,1,1),
         (0,1,1,1)]

true = [(1,0,0,0),
        (0,1,0,0),
        (0,0,1,0),
        (0,0,0,1)]

seed_dict_false = {i:1 for i in false}
seed_dict_true = {i:1 for i in true}

In [2]:
def create_left_dict(d):
    """
    convert a dictionary of 2x2's and return the counts on the left most numbers"""
    left = {(0,0):0,
            (1,0):0,
            (0,1):0,
            (1,1):0}
    for i in d:
        t = (i[0],i[2])
        left[t] += d[i]
    return left

def find_p_left(left, d):
    """
    left is a list of possible values (true or false - the global variables created at the beginning 
    that is being added to the left on the row. 
    d is a dictionary containing frequencies of the current left-most values (it will only have 4 keys). It can
    be crat
    Returns a new dictionary with frequencies"""
    new = {}    

    for i in left:
        t = (i[1], i[3])
        if t not in new:
            new[i] = 0
        if t in d:
            new[i] += d[t]            
            
    return new

def find_p_row(history):
    """
    The history is a dictionary in the form of a top/bot tuple dictionary
    returns one dictionary of all possible combinations for a row.  Use complete_to_top or complete_to_bottom 
    once this method is finished
    """
    length = len(history)
    complete = history.pop()
    bottom_row = {}
    while len(history) > 0:
        temp = complete
        complete = {}
        #find the overlaps between 2x2's and add to dictionary
        for i in temp.keys():
            right = (i[0][-1], i[1][-1])
            for j in history[-1].keys():
                left = (j[0][0],j[1][0])
                if right == left:
                    top = list(i[0]) + [j[0][1]]
                    bot = list(i[1]) + [j[1][1]]
                    complete[(tuple(top),tuple(bot))] = history[-1][j]
        history.pop()

    return complete

def complete_to_top(complete):
    top = {}
    for k,v in complete.items():
        if k[0] not in top:
            top[k[0]] = v
        else:
            top[k[0]] += v
    return top

def complete_to_bot(complete):
    bot = {}
    for k,v in complete.items():
        if k[1] not in bot:
            bot[k[1]] = {k[0]:v}
        else:
            bot[k[1]][k[0]] = v
    return bot

def top_bot_dict(d):
    """
    Make the dictionary a top/bottom dictionary
    e.g. (1,0,0,1) --> ((1,0),(0,1))"""
    new = {}
    for i in d.keys():
        new[((i[0],i[1]),(i[2],i[3]))] = d[i]
    return new

def match_bot_to_top(top,bot):
    """
    take a top dictionary from a lower row and match it with a bot dictionary from the row above it
    Return a new top dictionary with updated frequencies"""
    new = {}
    for k,v in top.items():
        if k in bot:
            for i in bot[k].keys():
                if i not in new:
                    new[i] = v
                else:
                    new[i] += v
    return new

def create_row(arr, d):
    """
    create a complete row.
    true / false are global variables"""
    history = [top_bot_dict(d)]
    while len(arr) > 0:
        left = create_left_dict(d)
        start = arr.pop()
        if start == 1:
            d = find_p_left(true,left)
        else:
            d = find_p_left(false,left)
        history.append(top_bot_dict(d))
    return history

In [11]:
def transpose(arr):
    new = []
    for i in range(len(arr[0])):
        row = []
        for j in arr:
            row.append(j[i])
        new.append(row)
    return new

In [12]:
input_arr = [[1,1,0,1,0,1,0,1,1,0],
             [1,1,0,0,0,0,1,1,1,0],
             [1,1,0,0,0,0,0,0,0,1],
             [0,1,0,0,0,0,1,1,0,0]]
transpose(input_arr)

[[1, 1, 1, 0],
 [1, 1, 1, 1],
 [0, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 1],
 [1, 1, 0, 1],
 [1, 1, 0, 0],
 [0, 0, 1, 0]]

## Create A Row:
take the current left-most dictionary and run it through the method create_left_dict.  This method will get you the left most value frequencies of the row.  Then we pass that dictionary and the true/false list of tuples.  This will discover how many possible new combinations we can have on our row and return a dictionary containing a new set of left most tuples and their frequencies

In [13]:
#input_arr = [[1,0,1],
#             [0,1,0],
#             [1,0,1]]
input_arr = [[1,1,0,1,0,1,0,1,1,0],
             [1,1,0,0,0,0,1,1,1,0],
             [1,1,0,0,0,0,0,0,0,1],
             [0,1,0,0,0,0,1,1,0,0]]
if len(input_arr) < len(input_arr[0]):
    input_arr = transpose(input_arr)
rows = []
while len(input_arr) > 0:
    #create all rows
    row = input_arr.pop()
    start = row.pop()
    d = {}
    if start == 1:
        d = seed_dict_true
    else:
        d = seed_dict_false

    row_history = create_row(row,d)
    complete = find_p_row(row_history.copy())
    rows.append(complete.copy())
    
#compare and aggregate all rows   
top = complete_to_top(rows.pop())
while len(rows) > 0:
    next_bottom = complete_to_bot(rows.pop())
    combined = match_bot_to_top(top, next_bottom)
    top = combined.copy()
print(sum(top.values()),"solutions: ", top)    


11567 solutions:  {(1, 1, 0, 0, 1): 470, (0, 0, 0, 1, 0): 884, (0, 0, 0, 1, 1): 884, (1, 1, 1, 0, 1): 1687, (0, 0, 0, 0, 1): 1113, (0, 0, 0, 0, 0): 671, (0, 1, 0, 1, 0): 16, (0, 1, 0, 1, 1): 38, (0, 1, 1, 0, 1): 803, (1, 0, 1, 0, 1): 16, (1, 1, 0, 1, 0): 16, (1, 1, 0, 1, 1): 38, (0, 1, 0, 0, 1): 246, (0, 1, 0, 0, 0): 82, (1, 1, 0, 0, 0): 82, (1, 0, 0, 1, 0): 787, (1, 0, 0, 1, 1): 787, (1, 0, 0, 0, 1): 1834, (1, 0, 0, 0, 0): 1047, (0, 1, 1, 0, 0): 22, (1, 0, 1, 0, 0): 22, (1, 1, 1, 0, 0): 22}
